# Comparison of Classification methods

Author: Dr. Vijesh J. Bhute   

<b>Highlights of this notebook:</b>
<ul><li>Performs classification of a dataset with more than 2 features.</li>
    <li> Compares different methods using Accuracy and Cumulative Accuracy Profile analysis. </li></ul>

<b>Types of classification models:</b>
<ul><li>Logistical regression</li>
    <li>K-Nearest Neighbours</li>
    <li>Support vector machine (SVM)</li>
    <li>Kernel SVM</li>
    <li>Naive Bayes</li>
    <li>Decision Tree</li>
    <li>Random Forest</li>
    <li>XGBoost</li>
    <li>CatBoost</li>
    <li>ANN</li>
    </ul> 

### Introduction: Important metrics for classification models

#### Accuracy and Confusion matrix 

For classification problems, accuracy for the model is given by, $$\text{Accuracy} =\frac{\text{TN+TP}}{\text{TN+FN+TP+FP}}$$ 

#### Confusion matrix

You can also look at the confusion matrix to see a summary of how the model has performed:
<table>
  <tr>
    <td></td>
    <th>Predicted No ($0$)</th>
    <th>Predicted Yes ($1$)</th>
  </tr>
  <tr>
    <th>Actual No ($0$)</th>
    <td>TN</td>
    <td>FP</td>
  </tr>
  <tr>
    <th>Actual Yes ($1$)</th>
    <td>FN</td>
    <td>TP</td>
  </tr>
</table>

### Importing the libraries

In [24]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
from sklearn.metrics import auc, confusion_matrix, accuracy_score

### Defining functions to streamline several computations

#### CAP analysis: Metric for comparing models

CAP: Cumulative Accuracy Profile is often used to compare models as accuracy alone may not be an ideal metric. 
<br><br>
This requires evaluating accuracy ratio. 
<br><br>
The first method to analyse the CAP Curve is using Area Under Curve. Let’s consider area under random model as $a$. We calculate the Accuracy Rate using the following steps:
<ul><li>Calculate the area under the perfect model ($a_P$) till the random model ($a$)</li>
    <li>Calculate the area under the prediction model ($a_R$) till the random model ($a$)</li>
    <li>Calculate Accuracy Ratio (AR) $= a_R / a_P$</li></ul>
The closer the Accuracy Ratio is to 1, better is the model.
<br> CAP analysis is adapted from <a href="https://www.kaggle.com/code/rohandawar/cap-cumulative-accuracy-profile-analysis-1/notebook" target="_blank">Kaggle website</a>

Below function finds the Accuracy ratio by comparing model prediction to random and perfect model. It takes the actual values and model predictions as input and returns the accuracy ratio which can be used to compare different models. <br>
<b>Main Assumption for using this function</b>: The actual values (target, y) takes binary values (0 and 1). 

In [25]:
def CAP_analysis(y_actual, y_predict):
    total = len(y_actual)
    one_count = np.sum(y_actual)
    
    lm = [y for _,y in sorted(zip(y_predict,y_actual), reverse=True)]
    xaxis = np.arange(0, total+1)
    yaxis = np.append([0], np.cumsum(lm))
    
    # Area under Random Model
    a = auc([0, total], [0, one_count])

    # Area between Perfect and Random Model
    aP = auc([0, one_count, total], [0, one_count, one_count]) - a

    # Area between Trained and Random Model
    aR = auc(xaxis, yaxis) - a

    return aR / aP

### Importing the dataset

I am going to use a breast cancer dataset from UCI for comparing different models. 

In [26]:
dataset = pd.read_csv('data/Data_UCI_Breast cancer_classification.csv')
dataset.head()

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


<ul><li>Last column is a binary variable. This is already encoded as 2 (benign) or 4 (malignant) and it might be beneficial to make this 0 and 1 by further encoding using Label Encoder.</li>
    <li>First column is not relevant for modelling and should be excluded from the training and test set</li></ul>

In [27]:
X = dataset.iloc[:, 1:-1].values
y = dataset.iloc[:, -1].values

### Optional: converting Class or target categorical variable

Encode target labels with value between 0 and n_classes-1.
<br>
This transformer should be used to encode target values or non-ordinal variables (such as gender)

In [28]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() # Use OrdinalEncoder if there is order in the variable
y= le.fit_transform (y)
#There is also ordinal encoder when variable is not only categorical but also has certain order (high, low for example)
sum(y)

239

### Optional: Converting non-ordinal categorical variables to multiple columns

Insert the index of the column that needs to be encoded. This column will be distrbuted to multiple columns taking binary values and moved to the front. 

In [29]:
#columnToBeEncoded = index 

#### One hot encoding

In [30]:
#from sklearn.compose import ColumnTransformer
#from sklearn.preprocessing import OneHotEncoder
#ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(),[columnToBeEncoded])] , remainder='passthrough')
##remainder is important otherwise all other columns will not be saved
#X_encoded = np.array(ct.fit_transform(X)) #np is used to convert the array to numpy array format 
##features should be in numpy array format for post processing and machine learning algorithms
#print(X_encoded)

### Splitting the dataset into the Training set and Test set

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

<b>Optional</b>: For large datasets, you can also split dataset into training, test, and validation set. You can train on training set, test on test set to assess overfitting and finally use validation set to compare multiple models. This can be done by using the below commands:

In [32]:
#from sklearn.model_selection import train_test_split
#X_2, X_test, y_2, y_test = train_test_split(X, y, test_size = 0.1, random_state = 0)
#X_train, X_validate, y_train, y_validate = train_test_split(X_2, y_2, test_size = 0.12, random_state = 0)
#print(len(X_train[:,1]))

### Optional: Feature scaling (Essential if performing PCA or other dimensionality reduction)

Feature scaling should be performed on the training set. 

<ul><li>Feature scaling should be done after splitting data into training and test set. Otherwise it would lead to leakage of data from test set to training set which is not realistic when using new data. </li>
    <li>Feature scaling is important when there is an implicit relationship between independent and dependent variable. </li>
    <li>Feature scaling is not necessary when the features are already in 0 to 1 (similar order of magnitude)</li>
<br>
</ul>
There are two main types of scaling:<ul>
<li>
<b> Standardisation</b>: Autoscaling (subtract mean and divide by standard deviation, brings values between roughly -3 and 3)</li>
<li><b> Normalisation </b>: Subtract min and divide by max-min (brings values in the range of 0 to 1) <br></li></ul>
<b>Note</b>: Normalisation can be used when features are normally distributed. 
Standardisation works all the time!

In [111]:
from sklearn.preprocessing import StandardScaler
scaling = True
if scaling:
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

<b>Note:</b> The predictors/features are of similar orders of magnitude. If they were different, then it would have been important to perform feature scaling (especially since the following models will relate to classification based on these features and there isn't any explicit relationship between $y$ and $x$). 
<br>
In this case, it is also fine to do the feature scaling as it will ensure that all features have similar average and standard deviation<br>

## Logistic Regression

Multiple linear regression model is given by, $$ y = b_0 + b_1 *x_1 + b_2 *x_2$$
<br>In logistic regression, you fit a function which lies in $0-1$. For example, sigmoid function, which is given by, $$p = \frac{1}{1+e^{-y}}$$
<br> Where $p$ is the probability which takes the values between $0$ and $1$. Solving above equation for $y$, we get, $$y = \ln \Big(\frac{p}{1-p}\Big) = b_0 + b_1*x_1 + b_2*x_2$$

In [112]:
from sklearn.linear_model import LogisticRegression
logModel = LogisticRegression(random_state = 0)
logModel.fit(X_train, y_train)
y_predict_logModel= logModel.predict(X_test) #Predicting test set
cm = confusion_matrix(y_test, y_predict_logModel)
print(cm)

[[84  3]
 [ 3 47]]


## K Nearest Neighbours

In [113]:
from sklearn.neighbors import KNeighborsClassifier
kNN = KNeighborsClassifier(n_neighbors=5, metric = 'minkowski',p=2) #minkowski with p=2 corresponds to Euclidean distance
kNN.fit(X_train, y_train)
y_predict_kNN=kNN.predict(X_test)
cm = confusion_matrix(y_test, y_predict_kNN)
print(cm)

[[83  4]
 [ 2 48]]


C:\Users\vbhute\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


## Support Vector Machine (SVM)

### Linear kernel for SVM

In [114]:
from sklearn.svm import SVC
LinearSVMmodel = SVC(kernel='linear', random_state = 0)
LinearSVMmodel.fit(X_train, y_train)
y_predict_LinSVM=LinearSVMmodel.predict(X_test)
cm = confusion_matrix(y_test,y_predict_LinSVM)
print(cm)

[[83  4]
 [ 2 48]]


### Kernel SVM

Types of common kernel functions:<br>
<ul><li>Gaussian Radial basis function (RBF)</li>
    <li>Sigmoid kernel </li>
    <li>Polynomial kernel</li>
    </ul>

In [115]:
from sklearn.svm import SVC
kSVMmodel = SVC() #default kernel is rbf (radial basis function)
#Other kernels include 'linear', ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’ or callable, default=’rbf’
kSVMmodel.fit(X_train, y_train)
y_predict_kSVM=kSVMmodel.predict(X_test)
cm = confusion_matrix(y_test,y_predict_kSVM)
print(cm)

[[82  5]
 [ 1 49]]


## Naive Bayes

<ul><li>Naive Bayes makes several assumptions but can be used even if the dataset don't satisfy these assumptions. </li><li>The most important assumption is that the <b>features are linearly independent of each other</b>. This is not true in most cases.</li> <li>
In spite of their apparently over-simplified assumptions, naive Bayes classifiers have worked quite well in many real-world situations, famously document classification and spam filtering. They require a small amount of training data to estimate the necessary parameters. </li>
    <li><b>This algorithm is good for classification but very good for estimation!</b></li>
<li>There are different types of Naive Bayes classifiers which differ mainly by the assumptions they make regarding the distribution of $P(x_i | y)$. These are:
<ul><li> Gaussian Naive Bayes</li>
    <li> Multinomial Naive Bayes </li>
    <li> Complement Naive Bayes</li>
    <li> Bernoulli Naive Bayes</li>
    <li> Categorical Naive Bayes </li></ul></li></ul>
You can learn more about Naive Bayes classifier from <a href="https://scikit-learn.org/stable/modules/naive_bayes.html" target="_blank">scikit learn website</a> <br>

### Gaussian Naive Bayes

In [116]:
from sklearn.naive_bayes import GaussianNB #MultinomialNB, ComplementNB, BernoulliNB, CategoricalNB
NBmodel = GaussianNB()
NBmodel.fit(X_train,y_train)
y_predict_NB= NBmodel.predict(X_test)
cm = confusion_matrix(y_test,y_predict_NB)
print(cm)

[[80  7]
 [ 0 50]]


### Decision Trees classifier

In [117]:
from sklearn.tree import DecisionTreeClassifier
DecTreeModel= DecisionTreeClassifier()
DecTreeModel.fit(X_train,y_train)
y_predict_DecTree= DecTreeModel.predict(X_test)
cm = confusion_matrix(y_test,y_predict_DecTree)
print(cm)

[[81  6]
 [ 1 49]]


## Random Forests classifier

In [118]:
from sklearn.ensemble import RandomForestClassifier
RFModel = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
RFModel.fit(X_train, y_train)
y_predict_RF= RFModel.predict(X_test)
cm = confusion_matrix(y_test,y_predict_RF)
print(cm)

[[84  3]
 [ 1 49]]


## XGBoost

In [119]:
#!pip install xgboost #Use this if xgboost is not already installed

In [120]:
from xgboost import XGBClassifier
XGBmodel = XGBClassifier()
XGBmodel.fit(X_train, y_train)
from sklearn.metrics import confusion_matrix, accuracy_score
y_predict_XGB = XGBmodel.predict(X_test)
cm = confusion_matrix(y_test, y_predict_XGB)
print(cm)
#accuracy_score(y_test, y_pred)

[16:57:50] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[[84  3]
 [ 1 49]]


C:\Users\vbhute\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


## CatBoost

In [121]:
#May need to install catboost via Anaconda using the cmd by using the following command: conda install catboost
from catboost import CatBoostClassifier
CatBoostmodel = CatBoostClassifier()
CatBoostmodel.fit(X_train, y_train)
from sklearn.metrics import confusion_matrix, accuracy_score
y_predict_CatB = CatBoostmodel.predict(X_test)
cm = confusion_matrix(y_test, y_predict_CatB)
print(cm)
#accuracy_score(y_test, y_pred)

Learning rate set to 0.007956
0:	learn: 0.6773044	total: 6.19ms	remaining: 6.18s
1:	learn: 0.6597970	total: 11.6ms	remaining: 5.78s
2:	learn: 0.6444872	total: 17.3ms	remaining: 5.74s
3:	learn: 0.6292693	total: 22.6ms	remaining: 5.63s
4:	learn: 0.6167804	total: 28.8ms	remaining: 5.72s
5:	learn: 0.6006776	total: 34ms	remaining: 5.64s
6:	learn: 0.5863011	total: 39.4ms	remaining: 5.59s
7:	learn: 0.5719487	total: 45.1ms	remaining: 5.59s
8:	learn: 0.5584160	total: 50.6ms	remaining: 5.58s
9:	learn: 0.5444026	total: 55.9ms	remaining: 5.54s
10:	learn: 0.5324566	total: 61.4ms	remaining: 5.52s
11:	learn: 0.5230008	total: 65.8ms	remaining: 5.42s
12:	learn: 0.5107659	total: 71.1ms	remaining: 5.4s
13:	learn: 0.4993509	total: 76.3ms	remaining: 5.38s
14:	learn: 0.4890183	total: 81.5ms	remaining: 5.35s
15:	learn: 0.4798578	total: 86ms	remaining: 5.29s
16:	learn: 0.4688945	total: 91.2ms	remaining: 5.28s
17:	learn: 0.4603259	total: 96.6ms	remaining: 5.27s
18:	learn: 0.4500329	total: 102ms	remaining: 5.27

216:	learn: 0.0616884	total: 472ms	remaining: 1.7s
217:	learn: 0.0614350	total: 473ms	remaining: 1.7s
218:	learn: 0.0611863	total: 475ms	remaining: 1.7s
219:	learn: 0.0608498	total: 477ms	remaining: 1.69s
220:	learn: 0.0605841	total: 479ms	remaining: 1.69s
221:	learn: 0.0604820	total: 480ms	remaining: 1.68s
222:	learn: 0.0602498	total: 481ms	remaining: 1.68s
223:	learn: 0.0599735	total: 483ms	remaining: 1.67s
224:	learn: 0.0597267	total: 485ms	remaining: 1.67s
225:	learn: 0.0594461	total: 486ms	remaining: 1.66s
226:	learn: 0.0593373	total: 487ms	remaining: 1.66s
227:	learn: 0.0591122	total: 489ms	remaining: 1.66s
228:	learn: 0.0588429	total: 491ms	remaining: 1.65s
229:	learn: 0.0586367	total: 493ms	remaining: 1.65s
230:	learn: 0.0583802	total: 494ms	remaining: 1.65s
231:	learn: 0.0582520	total: 496ms	remaining: 1.64s
232:	learn: 0.0580368	total: 497ms	remaining: 1.64s
233:	learn: 0.0578429	total: 499ms	remaining: 1.63s
234:	learn: 0.0576514	total: 501ms	remaining: 1.63s
235:	learn: 0.0

421:	learn: 0.0325055	total: 796ms	remaining: 1.09s
422:	learn: 0.0324412	total: 798ms	remaining: 1.09s
423:	learn: 0.0323503	total: 800ms	remaining: 1.09s
424:	learn: 0.0322575	total: 802ms	remaining: 1.08s
425:	learn: 0.0321747	total: 803ms	remaining: 1.08s
426:	learn: 0.0321282	total: 805ms	remaining: 1.08s
427:	learn: 0.0320523	total: 806ms	remaining: 1.08s
428:	learn: 0.0319740	total: 808ms	remaining: 1.07s
429:	learn: 0.0318741	total: 809ms	remaining: 1.07s
430:	learn: 0.0317971	total: 811ms	remaining: 1.07s
431:	learn: 0.0317438	total: 813ms	remaining: 1.07s
432:	learn: 0.0316463	total: 814ms	remaining: 1.07s
433:	learn: 0.0315492	total: 816ms	remaining: 1.06s
434:	learn: 0.0314618	total: 818ms	remaining: 1.06s
435:	learn: 0.0314036	total: 820ms	remaining: 1.06s
436:	learn: 0.0313509	total: 822ms	remaining: 1.06s
437:	learn: 0.0312688	total: 824ms	remaining: 1.06s
438:	learn: 0.0312315	total: 826ms	remaining: 1.05s
439:	learn: 0.0311244	total: 827ms	remaining: 1.05s
440:	learn: 

640:	learn: 0.0203529	total: 1.12s	remaining: 628ms
641:	learn: 0.0203049	total: 1.12s	remaining: 626ms
642:	learn: 0.0202647	total: 1.12s	remaining: 624ms
643:	learn: 0.0202332	total: 1.13s	remaining: 622ms
644:	learn: 0.0201969	total: 1.13s	remaining: 620ms
645:	learn: 0.0201611	total: 1.13s	remaining: 618ms
646:	learn: 0.0201343	total: 1.13s	remaining: 616ms
647:	learn: 0.0200822	total: 1.13s	remaining: 614ms
648:	learn: 0.0200387	total: 1.13s	remaining: 613ms
649:	learn: 0.0200015	total: 1.13s	remaining: 611ms
650:	learn: 0.0199703	total: 1.14s	remaining: 609ms
651:	learn: 0.0199467	total: 1.14s	remaining: 607ms
652:	learn: 0.0199259	total: 1.14s	remaining: 605ms
653:	learn: 0.0198938	total: 1.14s	remaining: 603ms
654:	learn: 0.0198459	total: 1.14s	remaining: 601ms
655:	learn: 0.0197902	total: 1.14s	remaining: 599ms
656:	learn: 0.0197456	total: 1.14s	remaining: 597ms
657:	learn: 0.0197147	total: 1.15s	remaining: 595ms
658:	learn: 0.0196653	total: 1.15s	remaining: 594ms
659:	learn: 

873:	learn: 0.0133679	total: 1.45s	remaining: 209ms
874:	learn: 0.0133340	total: 1.45s	remaining: 207ms
875:	learn: 0.0133061	total: 1.45s	remaining: 206ms
876:	learn: 0.0132805	total: 1.45s	remaining: 204ms
877:	learn: 0.0132623	total: 1.46s	remaining: 202ms
878:	learn: 0.0132331	total: 1.46s	remaining: 201ms
879:	learn: 0.0132012	total: 1.46s	remaining: 199ms
880:	learn: 0.0131890	total: 1.46s	remaining: 197ms
881:	learn: 0.0131456	total: 1.46s	remaining: 195ms
882:	learn: 0.0131259	total: 1.46s	remaining: 194ms
883:	learn: 0.0130856	total: 1.46s	remaining: 192ms
884:	learn: 0.0130671	total: 1.47s	remaining: 190ms
885:	learn: 0.0130541	total: 1.47s	remaining: 189ms
886:	learn: 0.0130332	total: 1.47s	remaining: 187ms
887:	learn: 0.0129962	total: 1.47s	remaining: 185ms
888:	learn: 0.0129745	total: 1.47s	remaining: 184ms
889:	learn: 0.0129550	total: 1.47s	remaining: 182ms
890:	learn: 0.0129362	total: 1.47s	remaining: 180ms
891:	learn: 0.0129121	total: 1.48s	remaining: 179ms
892:	learn: 

### ANN

In [129]:
import tensorflow as tf
tf.__version__

'2.10.0'

In [130]:
# Initializing the ANN

annModel = tf.keras.models.Sequential() #initialise the neural network framework

# Building hidden layer

annModel.add(tf.keras.layers.Dense(units = 6, activation = 'relu')) 
#6 neurons in this hidden layer, rectifier activation function

# Building 2nd hidden layer

annModel.add(tf.keras.layers.Dense(units = 6, activation = 'relu')) 
#6 neurons in this hidden layer, rectifier activation function

# Adding output layer

annModel.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid')) 
#1 neuron for binary prediction, sigmoid activation function to get a final choice of 0 or 1
#For non-binary classification, activation should be softmax

# Training ANN

# Compile ANN

annModel.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) 
#adam can perform stochastic gradient descent (update weights in each iteration to minimise the loss(error))
#binary or categorical cross entropy as the loss function
#Multiple metrics can be chosen, hence list is used

# Training

annModel.fit(X_train, y_train, batch_size = 32, epochs = 100)
#batch size is the number of entries which are used in batch learning

Epoch 1/100
18/18 [==============================] - 1s 2ms/step - loss: 0.9955 - accuracy: 0.0916
Epoch 2/100
18/18 [==============================] - 0s 3ms/step - loss: 0.8787 - accuracy: 0.1978
Epoch 3/100
18/18 [==============================] - 0s 3ms/step - loss: 0.8026 - accuracy: 0.3938
Epoch 4/100
18/18 [==============================] - 0s 3ms/step - loss: 0.7484 - accuracy: 0.5971
Epoch 5/100
18/18 [==============================] - 0s 4ms/step - loss: 0.7059 - accuracy: 0.6868
Epoch 6/100
18/18 [==============================] - 0s 4ms/step - loss: 0.6706 - accuracy: 0.7161
Epoch 7/100
18/18 [==============================] - 0s 4ms/step - loss: 0.6360 - accuracy: 0.7418
Epoch 8/100
18/18 [==============================] - 0s 4ms/step - loss: 0.6003 - accuracy: 0.7747
Epoch 9/100
18/18 [==============================] - 0s 4ms/step - loss: 0.5633 - accuracy: 0.8223
Epoch 10/100
18/18 [==============================] - 0s 4ms/step - loss: 0.5211 - accuracy: 0.8352
Epoch 11/

18/18 [==============================] - 0s 3ms/step - loss: 0.0705 - accuracy: 0.9725
Epoch 84/100
18/18 [==============================] - 0s 3ms/step - loss: 0.0703 - accuracy: 0.9725
Epoch 85/100
18/18 [==============================] - 0s 4ms/step - loss: 0.0701 - accuracy: 0.9725
Epoch 86/100
18/18 [==============================] - 0s 4ms/step - loss: 0.0700 - accuracy: 0.9725
Epoch 87/100
18/18 [==============================] - 0s 6ms/step - loss: 0.0697 - accuracy: 0.9725
Epoch 88/100
18/18 [==============================] - 0s 4ms/step - loss: 0.0697 - accuracy: 0.9725
Epoch 89/100
18/18 [==============================] - 0s 4ms/step - loss: 0.0694 - accuracy: 0.9725
Epoch 90/100
18/18 [==============================] - 0s 4ms/step - loss: 0.0693 - accuracy: 0.9725
Epoch 91/100
18/18 [==============================] - 0s 3ms/step - loss: 0.0691 - accuracy: 0.9725
Epoch 92/100
18/18 [==============================] - 0s 3ms/step - loss: 0.0690 - accuracy: 0.9725
Epoch 93/100


In [131]:
def predict_binary(model, X, scaled, sc):
    if scaled == 0:
        X = sc.transform(X)
    
    y_predict = model.predict(X)
    
    y_predict_binary = [0]*len(y_predict)
    for i in range(0,len(y_predict)):
        if(y_predict[i]>0.5):
            y_predict_binary[i] = 1
        else:
            y_predict_binary[i] = 0
    return y_predict_binary

In [135]:
y_predict_ANN_test = predict_binary(annModel, X_test, 1, sc)
y_predict_ANN_train = predict_binary(annModel, X_train, 1, sc)

18/18 [==============================] - 0s 3ms/step


### k-fold Cross-validation

In [122]:
kfold = 10

In [123]:
from sklearn.model_selection import cross_val_score
accuraciesLog = cross_val_score(estimator = logModel, X=X_train, y=y_train, cv=kfold)
accuracieskNN = cross_val_score(estimator = kNN, X=X_train, y=y_train, cv=kfold)
accuraciesLSVM = cross_val_score(estimator = LinearSVMmodel, X=X_train, y=y_train, cv=kfold)
accuraciesKSVM = cross_val_score(estimator = kSVMmodel, X=X_train, y=y_train, cv=kfold)
accuraciesNB = cross_val_score(estimator = NBmodel, X=X_train, y=y_train, cv=kfold)
accuraciesDT = cross_val_score(estimator = DecTreeModel, X=X_train, y=y_train, cv=kfold)
accuraciesRF = cross_val_score(estimator = RFModel, X=X_train, y=y_train, cv=kfold)
accuraciesXGB = cross_val_score(estimator = XGBmodel, X=X_train, y=y_train, cv=kfold)
accuraciesCatB = cross_val_score(estimator = CatBoostmodel, X=X_train, y=y_train, cv=kfold)
#print("Accuracy: {:.2f} %".format(accuraciesLog.mean()*100))
#print("Standard Deviation: {:.2f} %".format(accuraciesLog.std()*100))

C:\Users\vbhute\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\vbhute\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepd

[16:58:06] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:58:07] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\vbhute\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\vbhute\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:58:07] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:58:07] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\vbhute\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\vbhute\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

121:	learn: 0.1144457	total: 208ms	remaining: 1.5s
122:	learn: 0.1135413	total: 210ms	remaining: 1.49s
123:	learn: 0.1123577	total: 211ms	remaining: 1.49s
124:	learn: 0.1110794	total: 213ms	remaining: 1.49s
125:	learn: 0.1101813	total: 215ms	remaining: 1.49s
126:	learn: 0.1094830	total: 216ms	remaining: 1.49s
127:	learn: 0.1087747	total: 218ms	remaining: 1.48s
128:	learn: 0.1077136	total: 220ms	remaining: 1.48s
129:	learn: 0.1067335	total: 221ms	remaining: 1.48s
130:	learn: 0.1060822	total: 223ms	remaining: 1.48s
131:	learn: 0.1052600	total: 225ms	remaining: 1.48s
132:	learn: 0.1043772	total: 227ms	remaining: 1.48s
133:	learn: 0.1037178	total: 228ms	remaining: 1.47s
134:	learn: 0.1030848	total: 230ms	remaining: 1.47s
135:	learn: 0.1022085	total: 232ms	remaining: 1.47s
136:	learn: 0.1011995	total: 233ms	remaining: 1.47s
137:	learn: 0.1003366	total: 235ms	remaining: 1.47s
138:	learn: 0.0995858	total: 237ms	remaining: 1.47s
139:	learn: 0.0989793	total: 239ms	remaining: 1.47s
140:	learn: 0

315:	learn: 0.0457071	total: 529ms	remaining: 1.15s
316:	learn: 0.0456065	total: 531ms	remaining: 1.14s
317:	learn: 0.0454746	total: 533ms	remaining: 1.14s
318:	learn: 0.0453378	total: 535ms	remaining: 1.14s
319:	learn: 0.0452007	total: 537ms	remaining: 1.14s
320:	learn: 0.0450347	total: 538ms	remaining: 1.14s
321:	learn: 0.0448932	total: 540ms	remaining: 1.14s
322:	learn: 0.0446978	total: 541ms	remaining: 1.13s
323:	learn: 0.0445399	total: 543ms	remaining: 1.13s
324:	learn: 0.0443855	total: 545ms	remaining: 1.13s
325:	learn: 0.0442471	total: 547ms	remaining: 1.13s
326:	learn: 0.0441131	total: 548ms	remaining: 1.13s
327:	learn: 0.0439912	total: 550ms	remaining: 1.13s
328:	learn: 0.0438767	total: 552ms	remaining: 1.13s
329:	learn: 0.0437700	total: 553ms	remaining: 1.12s
330:	learn: 0.0436048	total: 555ms	remaining: 1.12s
331:	learn: 0.0434290	total: 557ms	remaining: 1.12s
332:	learn: 0.0433084	total: 559ms	remaining: 1.12s
333:	learn: 0.0431919	total: 561ms	remaining: 1.12s
334:	learn: 

518:	learn: 0.0272030	total: 851ms	remaining: 789ms
519:	learn: 0.0271493	total: 853ms	remaining: 787ms
520:	learn: 0.0270449	total: 855ms	remaining: 786ms
521:	learn: 0.0269988	total: 856ms	remaining: 784ms
522:	learn: 0.0269522	total: 858ms	remaining: 782ms
523:	learn: 0.0269116	total: 859ms	remaining: 781ms
524:	learn: 0.0268456	total: 861ms	remaining: 779ms
525:	learn: 0.0267958	total: 862ms	remaining: 777ms
526:	learn: 0.0267138	total: 864ms	remaining: 775ms
527:	learn: 0.0266616	total: 865ms	remaining: 774ms
528:	learn: 0.0266044	total: 867ms	remaining: 772ms
529:	learn: 0.0265191	total: 868ms	remaining: 770ms
530:	learn: 0.0264597	total: 870ms	remaining: 768ms
531:	learn: 0.0264441	total: 871ms	remaining: 766ms
532:	learn: 0.0263488	total: 872ms	remaining: 764ms
533:	learn: 0.0262879	total: 874ms	remaining: 762ms
534:	learn: 0.0262456	total: 875ms	remaining: 761ms
535:	learn: 0.0262125	total: 877ms	remaining: 759ms
536:	learn: 0.0261756	total: 879ms	remaining: 758ms
537:	learn: 

726:	learn: 0.0182018	total: 1.17s	remaining: 441ms
727:	learn: 0.0181922	total: 1.18s	remaining: 439ms
728:	learn: 0.0181477	total: 1.18s	remaining: 438ms
729:	learn: 0.0181088	total: 1.18s	remaining: 436ms
730:	learn: 0.0180795	total: 1.18s	remaining: 434ms
731:	learn: 0.0180585	total: 1.18s	remaining: 433ms
732:	learn: 0.0180266	total: 1.18s	remaining: 431ms
733:	learn: 0.0180026	total: 1.18s	remaining: 429ms
734:	learn: 0.0179525	total: 1.19s	remaining: 428ms
735:	learn: 0.0179165	total: 1.19s	remaining: 426ms
736:	learn: 0.0178907	total: 1.19s	remaining: 424ms
737:	learn: 0.0178570	total: 1.19s	remaining: 423ms
738:	learn: 0.0178290	total: 1.19s	remaining: 421ms
739:	learn: 0.0177973	total: 1.19s	remaining: 420ms
740:	learn: 0.0177619	total: 1.2s	remaining: 418ms
741:	learn: 0.0177233	total: 1.2s	remaining: 416ms
742:	learn: 0.0176764	total: 1.2s	remaining: 415ms
743:	learn: 0.0176456	total: 1.2s	remaining: 413ms
744:	learn: 0.0176158	total: 1.2s	remaining: 411ms
745:	learn: 0.017

942:	learn: 0.0125928	total: 1.5s	remaining: 90.6ms
943:	learn: 0.0125744	total: 1.5s	remaining: 89ms
944:	learn: 0.0125532	total: 1.5s	remaining: 87.4ms
945:	learn: 0.0125345	total: 1.5s	remaining: 85.8ms
946:	learn: 0.0125181	total: 1.5s	remaining: 84.2ms
947:	learn: 0.0124937	total: 1.5s	remaining: 82.6ms
948:	learn: 0.0124699	total: 1.51s	remaining: 81ms
949:	learn: 0.0124538	total: 1.51s	remaining: 79.4ms
950:	learn: 0.0124360	total: 1.51s	remaining: 77.8ms
951:	learn: 0.0124146	total: 1.51s	remaining: 76.2ms
952:	learn: 0.0123967	total: 1.51s	remaining: 74.6ms
953:	learn: 0.0123735	total: 1.51s	remaining: 73ms
954:	learn: 0.0123477	total: 1.52s	remaining: 71.4ms
955:	learn: 0.0123318	total: 1.52s	remaining: 69.8ms
956:	learn: 0.0122969	total: 1.52s	remaining: 68.3ms
957:	learn: 0.0122795	total: 1.52s	remaining: 66.7ms
958:	learn: 0.0122604	total: 1.52s	remaining: 65.1ms
959:	learn: 0.0122435	total: 1.52s	remaining: 63.5ms
960:	learn: 0.0122318	total: 1.52s	remaining: 61.9ms
961:	

## Model comparison

In [138]:
datAccuracyTraining = np.array([logModel.score(X_train, y_train), kNN.score(X_train, y_train), 
                       LinearSVMmodel.score(X_train,y_train), kSVMmodel.score(X_train,y_train), 
                       NBmodel.score(X_train,y_train), DecTreeModel.score(X_train, y_train), 
                       RFModel.score(X_train,y_train), XGBmodel.score(X_train,y_train), 
                       CatBoostmodel.score(X_train,y_train), accuracy_score(y_train, y_predict_ANN_train)])

datAccuracyTest = np.array([logModel.score(X_test, y_test), kNN.score(X_test,y_test), 
                   LinearSVMmodel.score(X_test,y_test), kSVMmodel.score(X_test,y_test), 
                   NBmodel.score(X_test,y_test), DecTreeModel.score(X_test, y_test),
                   RFModel.score(X_test,y_test), XGBmodel.score(X_test,y_test), 
                   CatBoostmodel.score(X_test,y_test), accuracy_score(y_test, y_predict_ANN_test)])

#datCAP =  np.array([CAP_analysis(y_test, y_predict_logModel), CAP_analysis(y_test, y_predict_kNN), 
#          CAP_analysis(y_test, y_predict_LinSVM), CAP_analysis(y_test, y_predict_kSVM),
#          CAP_analysis(y_test, y_predict_NB), CAP_analysis(y_test, y_predict_DecTree),
#          CAP_analysis(y_test, y_predict_RF), CAP_analysis(y_test, y_predict_XGB),
#          CAP_analysis(y_test, y_predict_CatB)])

crossVal_Accuracy =  np.array([accuraciesLog.max(), accuracieskNN.max(), accuraciesLSVM.max(), 
                     accuraciesNB.max(), accuraciesDT.max(), accuraciesRF.max(), 
                     accuraciesKSVM.max(), accuraciesXGB.max(), accuraciesCatB.max(), 0])

crossVal_std =  np.array([accuraciesLog.std(), accuracieskNN.std(), accuraciesLSVM.std(), 
                accuraciesNB.std(), accuraciesDT.std(), accuraciesRF.std(), 
                accuraciesKSVM.std(),accuraciesXGB.std(), accuraciesCatB.std(), 0])

#Round and conveert to percentage
datAccuracyTraining = np.around(datAccuracyTraining*100, decimals = 2)
datAccuracyTest = np.around(datAccuracyTest*100, decimals = 2)
datCAP = np.around(datCAP*100, decimals = 2)
crossVal_Accuracy = np.around(crossVal_Accuracy*100, decimals =2)
crossVal_std = np.around(crossVal_std*100, decimals = 2 )

C:\Users\vbhute\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\vbhute\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepd

In [140]:
dat = {'Model name':['Log Regression', 'kNN', 
                     'Linear SVM', 'Kernel SVM', 
                     'Naive Bayes', 'Decision Trees', 
                     'Random Forests', 'XGBoost', 'CatBoost', 'ANN'], 
       'Training set Accuracy':datAccuracyTraining, 
       'Test set Accuracy':datAccuracyTest, 
       #'Accuracy Ratio (CAP)': datCAP, 
       'Cross-validation Accuracy (max)': crossVal_Accuracy, 
       'Cross-validation Accuracy (std)': crossVal_std}
accuracyDF = pd.DataFrame(data = dat)
accuracyDF.sort_values('Training set Accuracy', ascending = False)

,Model name,Training set Accuracy,Test set Accuracy,Cross-validation Accuracy (max),Cross-validation Accuracy (std)
5,Decision Trees,100.00,94.89,100.0,2.30
6,Random Forests,100.00,97.08,100.0,2.17
7,XGBoost,100.00,97.08,100.0,2.17
8,CatBoost,100.00,97.81,100.0,2.50
9,ANN,97.62,95.62,0.0,0.00
0,Log Regression,97.44,95.62,100.0,1.97
2,Linear SVM,97.44,95.62,100.0,2.19
1,kNN,97.25,95.62,100.0,1.79
3,Kernel SVM,97.25,95.62,100.0,2.24
4,Naive Bayes,96.52,94.89,100.0,2.64


The dataset is relatively small and ANN is not necessarily needed as other classfication models which are more interpretable work better in classifying patients. 